# IPC Database Builder - 2025.01 Edition

This notebook processes the latest WIPO IPC XML file (EN_ipc_scheme_20250101.xml) and builds an improved database structure for the IPC Browser application.

## Features:
- Parse IPC 2025.01 XML structure
- Extract hierarchical classification data
- Calculate statistics and relationships
- Create optimized SQLite database
- Maintain backward compatibility with existing visualization code

In [35]:
#!/usr/bin/env python3
"""
IPC Database Builder for WIPO IPC 2025.01

This script processes the latest IPC XML file and creates a comprehensive database
for the IPC Browser visualization application.
"""

import sqlite3
import pandas as pd
from lxml import etree as ET
import numpy as np
import re
import time
from datetime import datetime
from pathlib import Path
import warnings
from typing import Dict, List, Tuple, Optional
warnings.filterwarnings('ignore')

print("🔧 IPC Database Builder - 2025.01 Edition")
print("=" * 50)

🔧 IPC Database Builder - 2025.01 Edition


## 1. Configuration and Setup

In [36]:
# Configuration
class IPCConfig:
    """
    Configuration class for IPC database processing
    """
    
    # File paths
    IPC_XML_FILE = '/home/jovyan/mtc-patent-analytics/ipc-browser/ipc/EN_ipc_scheme_20250101.xml'
    OUTPUT_DB = '/home/jovyan/mtc-patent-analytics/ipc-browser/patent-classification-2025.db'
        
    # IPC version info
    IPC_VERSION = '2025.01'
    IPC_EDITION = '20250101'
    IPC_LANGUAGE = 'EN'
    
    # XML namespaces
    NAMESPACES = {
        'ipc': 'http://www.wipo.int/classifications/ipc/masterfiles',
        'xhtml': 'http://www.w3.org/1999/xhtml'
    }
    
    # Kind to level mapping (from old system)
    KIND_TO_LEVEL = {
        's': 2,  # Section
        'c': 3,  # Class  
        'u': 4,  # Subclass
        'm': 5,  # Main group
        '1': 6,  # 1-dot group
        '2': 7,  # 2-dot group  
        '3': 8,  # 3-dot group
        '4': 9,  # 4-dot group
        '5': 10, # 5-dot group
        '6': 11, # 6-dot group
        '7': 12, # 7-dot group
        '8': 13, # 8-dot group
        '9': 14, # 9-dot group
        'A': 15, # A-dot group
        'B': 16, # B-dot group
        'C': 17  # C-dot group
    }
    
    # Default creation date for sections (IPC started)
    DEFAULT_CREATION_DATE = 19680901

config = IPCConfig()
print(f"📁 Input file: {config.IPC_XML_FILE}")
print(f"🗄️  Output database: {config.OUTPUT_DB}")
print(f"📋 IPC Version: {config.IPC_VERSION}")

📁 Input file: /home/jovyan/mtc-patent-analytics/ipc-browser/ipc/EN_ipc_scheme_20250101.xml
🗄️  Output database: /home/jovyan/mtc-patent-analytics/ipc-browser/patent-classification-2025.db
📋 IPC Version: 2025.01


## 2. IPC XML Parser Class

In [37]:
class IPCXMLParser:
    """
    Parser for WIPO IPC XML files
    """
    
    def __init__(self, xml_file_path: str):
        self.xml_file_path = xml_file_path
        self.tree = None
        self.root = None
        self.namespaces = config.NAMESPACES
        
    def load_xml(self) -> bool:
        """
        Load and parse the XML file
        """
        try:
            print(f"📖 Loading XML file: {self.xml_file_path}")
            parser = ET.XMLParser(remove_blank_text=True)
            self.tree = ET.parse(self.xml_file_path, parser=parser)
            self.root = self.tree.getroot()
            
            # Get file info
            file_size = Path(self.xml_file_path).stat().st_size / (1024 * 1024)  # MB
            print(f"✓ XML loaded successfully ({file_size:.1f} MB)")
            print(f"✓ Root element: {self.root.tag}")
            print(f"✓ Edition: {self.root.get('edition')}")
            print(f"✓ Language: {self.root.get('lang')}")
            
            return True
            
        except Exception as e:
            print(f"❌ Error loading XML: {e}")
            return False
    
    def extract_title_text(self, title_element) -> str:
        """
        Extract and concatenate title text from title element
        """
        if title_element is None:
            return ""
            
        title_parts = []
        
        # Find all titlePart elements
        for title_part in title_element.findall('.//ipc:titlePart', self.namespaces):
            # Get text content, excluding references
            text_elem = title_part.find('ipc:text', self.namespaces)
            if text_elem is not None and text_elem.text:
                title_parts.append(text_elem.text.strip())
        
        return '; '.join(title_parts) if title_parts else ""
    
    def format_symbol(self, symbol: str) -> str:
        """
        Format IPC symbol for display (similar to old system)
        """
        if not symbol or len(symbol) <= 4:
            return symbol
        
        # For groups: remove leading/trailing zeros and add slash
        if len(symbol) > 4:
            try:
                # Format: H01F0001053000 -> H01F1/053
                base = symbol[:4]  # H01F
                main_group = str(int(symbol[4:8]))  # 0001 -> 1
                sub_group = symbol[8:].rstrip('0')  # 053000 -> 053
                
                if sub_group:
                    return f"{base}{main_group}/{sub_group}"
                else:
                    return f"{base}{main_group}/00"
                    
            except (ValueError, IndexError):
                return symbol
        
        return symbol
    
    def parse_edition_date(self, edition_str: str) -> int:
        """
        Parse edition string to integer date
        Example: '19680901,20060101' -> 19680901 (first date)
        """
        if not edition_str:
            return config.DEFAULT_CREATION_DATE
            
        # Take the first date if multiple dates exist
        first_date = edition_str.split(',')[0]
        try:
            return int(first_date)
        except ValueError:
            return config.DEFAULT_CREATION_DATE
    
    def extract_ipc_entries(self) -> List[Dict]:
        """
        Extract all IPC entries from the XML with hierarchical structure
        """
        print("🔍 Extracting IPC entries...")
        entries = []
        
        def process_entry(element, parent_symbol='IPC', level_offset=0):
            """
            Recursively process IPC entries
            """
            kind = element.get('kind')
            symbol = element.get('symbol')
            edition = element.get('edition', '')
            
            # Skip certain kinds (title, index, etc.)
            if kind in ['t', 'i', 'g', 'n']:
                # Process children but don't add this entry
                for child in element.findall('ipc:ipcEntry', self.namespaces):
                    process_entry(child, parent_symbol, level_offset)
                return
            
            if symbol and kind:
                # Extract title
                title_element = element.find('.//ipc:title', self.namespaces)
                title = self.extract_title_text(title_element)
                
                # Calculate level
                level = config.KIND_TO_LEVEL.get(kind, 2)
                
                # Parse creation date
                creation_date = self.parse_edition_date(edition)
                
                # Format symbols
                symbol_short = self.format_symbol(symbol)
                parent_short = self.format_symbol(parent_symbol) if parent_symbol != 'IPC' else parent_symbol
                
                entry = {
                    'symbol': symbol,
                    'kind': kind,
                    'parent': parent_symbol,
                    'level': level,
                    'symbol_short': symbol_short,
                    'parent_short': parent_short,
                    'title_en': title,
                    'creation_date': creation_date
                }
                
                entries.append(entry)
                
                # Process children with current symbol as parent
                for child in element.findall('ipc:ipcEntry', self.namespaces):
                    process_entry(child, symbol, level_offset)
        
        # Start processing from root
        for entry in self.root.findall('ipc:ipcEntry', self.namespaces):
            process_entry(entry)
        
        print(f"✓ Extracted {len(entries)} IPC entries")
        return entries

# Initialize parser
parser = IPCXMLParser(config.IPC_XML_FILE)
print("🔧 IPC XML Parser initialized")

🔧 IPC XML Parser initialized


## 3. Parse XML and Extract Data

In [38]:
# Load and parse XML
start_time = time.time()

if parser.load_xml():
    print("\n📊 Parsing IPC entries...")
    ipc_entries = parser.extract_ipc_entries()
    
    # Convert to DataFrame
    ipc_df = pd.DataFrame(ipc_entries)
    
    print(f"\n✅ Parsing completed in {time.time() - start_time:.2f} seconds")
    print(f"📈 Total entries: {len(ipc_df)}")
    
    # Show basic statistics
    print("\n📋 Entry distribution by level:")
    level_counts = ipc_df['level'].value_counts().sort_index()
    for level, count in level_counts.items():
        level_name = {2: 'Sections', 3: 'Classes', 4: 'Subclasses', 5: 'Main Groups'}.get(level, f'Level {level}')
        print(f"   Level {level} ({level_name}): {count:,} entries")
    
    # Show sample data
    print("\n🔍 Sample entries:")
    display(ipc_df.head(10))
else:
    print("❌ Failed to load XML file")

📖 Loading XML file: /home/jovyan/mtc-patent-analytics/ipc-browser/ipc/EN_ipc_scheme_20250101.xml
✓ XML loaded successfully (19.6 MB)
✓ Root element: {http://www.wipo.int/classifications/ipc/masterfiles}IPCScheme
✓ Edition: 20250101
✓ Language: EN

📊 Parsing IPC entries...
🔍 Extracting IPC entries...
✓ Extracted 79833 IPC entries

✅ Parsing completed in 1.87 seconds
📈 Total entries: 79833

📋 Entry distribution by level:
   Level 2 (Sections): 8 entries
   Level 3 (Classes): 132 entries
   Level 4 (Subclasses): 654 entries
   Level 5 (Main Groups): 7,630 entries
   Level 6 (Level 6): 24,045 entries
   Level 7 (Level 7): 24,075 entries
   Level 8 (Level 8): 14,301 entries
   Level 9 (Level 9): 6,166 entries
   Level 10 (Level 10): 1,992 entries
   Level 11 (Level 11): 629 entries
   Level 12 (Level 12): 151 entries
   Level 13 (Level 13): 47 entries
   Level 14 (Level 14): 3 entries

🔍 Sample entries:


,symbol,kind,parent,level,symbol_short,parent_short,title_en,creation_date
0,A,s,IPC,2,A,IPC,HUMAN NECESSITIES,19680901
1,A01,c,A,3,A01,A,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,19680901
2,A01B,u,A01,4,A01B,A01,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,19680901
3,A01B0001000000,m,A01B,5,A01B1/00,A01B,Hand tools,19680901
4,A01B0001020000,1,A01B0001000000,6,A01B1/02,A01B1/00,Spades; Shovels,19680901
5,A01B0001040000,2,A01B0001020000,7,A01B1/04,A01B1/02,with teeth,19680901
6,A01B0001060000,1,A01B0001000000,6,A01B1/06,A01B1/00,Hoes; Hand cultivators,19680901
7,A01B0001080000,2,A01B0001060000,7,A01B1/08,A01B1/06,with a single blade,19680901
8,A01B0001100000,2,A01B0001060000,7,A01B1/1,A01B1/06,with two or more blades,19680901
9,A01B0001120000,2,A01B0001060000,7,A01B1/12,A01B1/06,with blades provided with teeth,19680901


## 4. Calculate Statistics and Hierarchy

In [39]:
# Test the optimized statistics calculation directly
print("🧪 Testing Cell 10 - Statistics Calculation Optimization")
print("=" * 60)

class IPCStatisticsCalculator:
    """Calculate statistics for IPC classifications - OPTIMIZED VERSION"""
    
    def __init__(self, ipc_dataframe):
        self.df = ipc_dataframe.copy()
        
    def calculate_descendant_counts_optimized(self):
        """Calculate descendants using optimized bottom-up approach"""
        print("🧮 Calculating descendant counts (optimized)...")
        start_time = time.time()
        
        # Initialize all sizes to 0
        self.df['size'] = 0
        
        # Create lookup tables for O(1) access
        symbol_to_index = {symbol: idx for idx, symbol in enumerate(self.df['symbol'])}
        parent_to_children = {}
        
        # Build parent -> children mapping
        for _, row in self.df.iterrows():
            parent = row['parent']
            if parent not in parent_to_children:
                parent_to_children[parent] = []
            parent_to_children[parent].append(row['symbol'])
        
        print(f"   Built lookup tables for {len(self.df)} entries...")
        
        # Process bottom-up by level (highest to lowest)
        levels = sorted(self.df['level'].unique(), reverse=True)
        
        for level in levels:
            level_entries = self.df[self.df['level'] == level]
            print(f"   Processing level {level}: {len(level_entries)} entries...")
            
            for _, entry in level_entries.iterrows():
                symbol = entry['symbol']
                children = parent_to_children.get(symbol, [])
                
                # Count total descendants 
                total_descendants = len(children)
                for child in children:
                    if child in symbol_to_index:
                        child_idx = symbol_to_index[child]
                        total_descendants += self.df.iloc[child_idx]['size']
                
                # Update size
                entry_idx = symbol_to_index[symbol]
                self.df.iloc[entry_idx, self.df.columns.get_loc('size')] = total_descendants
        
        elapsed = time.time() - start_time
        print(f"✓ Completed in {elapsed:.2f} seconds")
        return self.df

    def calculate_percentages(self):
        """Calculate percentage distribution"""
        print("📊 Calculating percentages...")
        
        # Use total descendants of root sections
        root_total = self.df[self.df['level'] == 2]['size'].sum()
        
        if root_total > 0:
            self.df['size_percent'] = (self.df['size'] * 100 / root_total).round(3)
        else:
            self.df['size_percent'] = 0.0
        
        print(f"✓ Calculated percentages (base: {root_total:,} total descendants)")
        return self.df

    def calculate_normalized_sizes(self):
        """Calculate normalized sizes for visualization (3-13 scale)"""
        print("📏 Calculating normalized sizes...")
        
        try:
            from sklearn.preprocessing import MinMaxScaler
        except ImportError:
            print("   Warning: sklearn not available, using simple normalization")
            self.df['size_normalised'] = 8.0
            return self.df
        
        # Initialize with middle value
        self.df['size_normalised'] = 8.0
        
        # Normalize within each parent group
        grouped = self.df.groupby('parent')
        
        normalized_count = 0
        for parent, group in grouped:
            if len(group) > 1 and group['size'].max() > 0:
                sizes = group['size'].values.reshape(-1, 1)
                if sizes.max() > sizes.min():
                    scaler = MinMaxScaler(feature_range=(3, 13))
                    normalized = scaler.fit_transform(sizes).flatten()
                    self.df.loc[group.index, 'size_normalised'] = normalized
                    normalized_count += len(group)
        
        print(f"✓ Normalized {normalized_count} entries for visualization")
        return self.df

# Test with sample data first
if 'ipc_df' in locals():
    print(f"Testing with {len(ipc_df)} IPC entries...")
    
    # Test the optimization
    test_start = time.time()
    calculator = IPCStatisticsCalculator(ipc_df)
    
    # Run all statistics calculations
    print("\n⏱️  Step 1: Calculate descendant counts...")
    ipc_df = calculator.calculate_descendant_counts_optimized()
    
    print("\n⏱️  Step 2: Calculate percentages...")
    ipc_df = calculator.calculate_percentages()
    
    print("\n⏱️  Step 3: Calculate normalized sizes...")
    ipc_df = calculator.calculate_normalized_sizes()
    
    test_time = time.time() - test_start
    
    print(f"\n🎉 OPTIMIZATION TEST SUCCESSFUL!")
    print(f"⏱️  Total time: {test_time:.2f} seconds")
    print(f"📊 Processed {len(ipc_df):,} entries")
    print(f"🔢 Max descendants: {ipc_df['size'].max():,}")
    print(f"✅ Performance: {len(ipc_df) / test_time:.0f} entries/second")
    
    # Verify all required columns exist
    required_columns = ['size', 'size_percent', 'size_normalised']
    missing_columns = [col for col in required_columns if col not in ipc_df.columns]
    
    if missing_columns:
        print(f"⚠️  Missing columns: {missing_columns}")
    else:
        print(f"✅ All required columns present: {required_columns}")
    
    # Show column info
    print(f"\n📋 DataFrame columns: {list(ipc_df.columns)}")
    print(f"📊 DataFrame shape: {ipc_df.shape}")
    
else:
    print("⚠️  No ipc_df available - run previous cells first")

print("\n✨ Cell 10 optimization verified!")

🧪 Testing Cell 10 - Statistics Calculation Optimization
Testing with 79833 IPC entries...

⏱️  Step 1: Calculate descendant counts...
🧮 Calculating descendant counts (optimized)...
   Built lookup tables for 79833 entries...
   Processing level 14: 3 entries...
   Processing level 13: 47 entries...
   Processing level 12: 151 entries...
   Processing level 11: 629 entries...
   Processing level 10: 1992 entries...
   Processing level 9: 6166 entries...
   Processing level 8: 14301 entries...
   Processing level 7: 24075 entries...
   Processing level 6: 24045 entries...
   Processing level 5: 7630 entries...
   Processing level 4: 654 entries...
   Processing level 3: 132 entries...
   Processing level 2: 8 entries...
✓ Completed in 20.18 seconds

⏱️  Step 2: Calculate percentages...
📊 Calculating percentages...
✓ Calculated percentages (base: 79,825 total descendants)

⏱️  Step 3: Calculate normalized sizes...
📏 Calculating normalized sizes...
✓ Normalized 48149 entries for visualizat

## 5. Create Database

In [40]:
class IPCDatabaseBuilder:
    """
    Build optimized SQLite database for IPC data
    """
    
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.conn = None
        
    def create_database(self) -> bool:
        """
        Create database with optimized schema
        """
        try:
            print(f"🗄️  Creating database: {self.db_path}")
            
            # Remove existing database
            if Path(self.db_path).exists():
                Path(self.db_path).unlink()
                print("🗑️  Removed existing database")
            
            self.conn = sqlite3.connect(self.db_path)
            cursor = self.conn.cursor()
            
            # Create main IPC table (backward compatible with existing visualization)
            cursor.execute('''
                CREATE TABLE ipc (
                    symbol TEXT PRIMARY KEY,
                    kind TEXT NOT NULL,
                    parent TEXT NOT NULL,
                    level INTEGER NOT NULL,
                    symbol_short TEXT,
                    parent_short TEXT,
                    title_en TEXT,
                    title_fr TEXT,  -- Placeholder for future French titles
                    size INTEGER DEFAULT 0,
                    size_percent REAL DEFAULT 0.0,
                    size_normalised REAL DEFAULT 8.0,
                    creation_date INTEGER
                )
            ''')
            
            # Create metadata table
            cursor.execute('''
                CREATE TABLE ipc_metadata (
                    key TEXT PRIMARY KEY,
                    value TEXT
                )
            ''')
            
            # Create indexes for performance
            cursor.execute('CREATE INDEX idx_ipc_level ON ipc(level)')
            cursor.execute('CREATE INDEX idx_ipc_parent ON ipc(parent)')
            cursor.execute('CREATE INDEX idx_ipc_kind ON ipc(kind)')
            cursor.execute('CREATE INDEX idx_ipc_symbol_short ON ipc(symbol_short)')
            
            self.conn.commit()
            print("✓ Database schema created successfully")
            return True
            
        except Exception as e:
            print(f"❌ Error creating database: {e}")
            return False
    
    def insert_data(self, dataframe: pd.DataFrame) -> bool:
        """
        Insert IPC data into database
        """
        try:
            print(f"💾 Inserting {len(dataframe)} records into database...")
            
            # Ensure all required columns exist
            df = dataframe.copy()
            
            # Add missing columns with defaults if they don't exist
            if 'title_fr' not in df.columns:
                df['title_fr'] = None  # Placeholder for French titles
            
            if 'size' not in df.columns:
                df['size'] = 0
                print("   Added missing 'size' column")
                
            if 'size_percent' not in df.columns:
                df['size_percent'] = 0.0
                print("   Added missing 'size_percent' column")
                
            if 'size_normalised' not in df.columns:
                df['size_normalised'] = 8.0
                print("   Added missing 'size_normalised' column")
            
            # Debug: Show available columns
            print(f"   Available columns: {list(df.columns)}")
            
            # Select columns in correct order
            columns = [
                'symbol', 'kind', 'parent', 'level', 'symbol_short', 'parent_short',
                'title_en', 'title_fr', 'size', 'size_percent', 'size_normalised', 'creation_date'
            ]
            
            # Verify all columns exist before insertion
            missing_cols = [col for col in columns if col not in df.columns]
            if missing_cols:
                print(f"❌ Missing columns: {missing_cols}")
                return False
            
            # Insert data
            df[columns].to_sql('ipc', self.conn, if_exists='append', index=False)
            
            # Insert metadata
            metadata = [
                ('ipc_version', config.IPC_VERSION),
                ('ipc_edition', config.IPC_EDITION),
                ('ipc_language', config.IPC_LANGUAGE),
                ('created_at', datetime.now().isoformat()),
                ('total_entries', str(len(dataframe))),
                ('source_file', config.IPC_XML_FILE)
            ]
            
            cursor = self.conn.cursor()
            cursor.executemany('INSERT INTO ipc_metadata (key, value) VALUES (?, ?)', metadata)
            
            self.conn.commit()
            print("✓ Data inserted successfully")
            return True
            
        except Exception as e:
            print(f"❌ Error inserting data: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def verify_data(self) -> bool:
        """
        Verify database integrity
        """
        try:
            cursor = self.conn.cursor()
            
            # Check total count
            cursor.execute('SELECT COUNT(*) FROM ipc')
            total_count = cursor.fetchone()[0]
            
            # Check level distribution
            cursor.execute('SELECT level, COUNT(*) FROM ipc GROUP BY level ORDER BY level')
            level_distribution = cursor.fetchall()
            
            print(f"\n🔍 Database Verification:")
            print(f"   Total entries: {total_count:,}")
            print(f"   Level distribution:")
            for level, count in level_distribution:
                level_name = {2: 'Sections', 3: 'Classes', 4: 'Subclasses', 5: 'Main Groups'}.get(level, f'Level {level}')
                print(f"     Level {level} ({level_name}): {count:,}")
            
            # Check for orphaned entries
            cursor.execute('''
                SELECT COUNT(*) FROM ipc i1 
                WHERE i1.parent != 'IPC' 
                AND NOT EXISTS (SELECT 1 FROM ipc i2 WHERE i2.symbol = i1.parent)
            ''')
            orphaned_count = cursor.fetchone()[0]
            print(f"   Orphaned entries: {orphaned_count}")
            
            # Check statistics columns
            cursor.execute('SELECT AVG(size), AVG(size_percent), AVG(size_normalised) FROM ipc')
            stats = cursor.fetchone()
            print(f"   Statistics averages: size={stats[0]:.1f}, percent={stats[1]:.2f}%, normalized={stats[2]:.1f}")
            
            if orphaned_count == 0:
                print("✅ Database verification passed!")
                return True
            else:
                print("⚠️  Warning: Found orphaned entries")
                return False
                
        except Exception as e:
            print(f"❌ Error verifying database: {e}")
            return False
    
    def close(self):
        """
        Close database connection
        """
        if self.conn:
            self.conn.close()
            print("🔒 Database connection closed")

# Build database
print("\n🗄️  Building database...")

# First, verify that ipc_df has all required columns
if 'ipc_df' in locals():
    print(f"📊 Checking dataframe with {len(ipc_df)} entries...")
    print(f"   Columns available: {list(ipc_df.columns)}")
    
    # Ensure statistics columns exist (they should from cell 9)
    required_stats_cols = ['size', 'size_percent', 'size_normalised']
    missing_stats = [col for col in required_stats_cols if col not in ipc_df.columns]
    
    if missing_stats:
        print(f"⚠️  Missing statistics columns: {missing_stats}")
        print("   Running statistics calculation first...")
        
        # Run statistics calculation if missing
        calculator = IPCStatisticsCalculator(ipc_df)
        ipc_df = calculator.calculate_descendant_counts_optimized()
        ipc_df = calculator.calculate_percentages()
        ipc_df = calculator.calculate_normalized_sizes()
        
        print(f"✓ Statistics columns added: {[col for col in required_stats_cols if col in ipc_df.columns]}")
    else:
        print("✓ All statistics columns present")

    # Now create database
    db_builder = IPCDatabaseBuilder(config.OUTPUT_DB)

    if db_builder.create_database():
        if db_builder.insert_data(ipc_df):
            db_builder.verify_data()
            print(f"\n📊 Database ready: {config.OUTPUT_DB}")
        else:
            print("❌ Failed to insert data")
    else:
        print("❌ Failed to create database")
else:
    print("❌ ipc_df not found - run previous cells first")


🗄️  Building database...
📊 Checking dataframe with 79833 entries...
   Columns available: ['symbol', 'kind', 'parent', 'level', 'symbol_short', 'parent_short', 'title_en', 'creation_date', 'size', 'size_percent', 'size_normalised']
✓ All statistics columns present
🗄️  Creating database: /home/jovyan/mtc-patent-analytics/ipc-browser/patent-classification-2025.db
🗑️  Removed existing database
✓ Database schema created successfully
💾 Inserting 79833 records into database...
   Available columns: ['symbol', 'kind', 'parent', 'level', 'symbol_short', 'parent_short', 'title_en', 'creation_date', 'size', 'size_percent', 'size_normalised', 'title_fr']
✓ Data inserted successfully

🔍 Database Verification:
   Total entries: 79,833
   Level distribution:
     Level 2 (Sections): 8
     Level 3 (Classes): 132
     Level 4 (Subclasses): 654
     Level 5 (Main Groups): 7,630
     Level 6 (Level 6): 24,045
     Level 7 (Level 7): 24,075
     Level 8 (Level 8): 14,301
     Level 9 (Level 9): 6,166
 

## 6. Compatibility Testing

In [41]:
# Test compatibility with existing visualization code
print("🧪 Testing compatibility with existing visualization...")

try:
    # Test the same queries used by the visualization
    test_df = pd.read_sql_query("SELECT * FROM ipc LIMIT 10", db_builder.conn)
    
    print("✓ Basic query test passed")
    print(f"   Retrieved {len(test_df)} test records")
    
    # Check required columns
    required_columns = [
        'symbol', 'kind', 'parent', 'level', 'symbol_short', 'parent_short',
        'title_en', 'size', 'size_percent', 'size_normalised', 'creation_date'
    ]
    
    missing_columns = [col for col in required_columns if col not in test_df.columns]
    
    if not missing_columns:
        print("✓ All required columns present")
    else:
        print(f"❌ Missing columns: {missing_columns}")
    
    # Test hierarchy queries
    sections_df = pd.read_sql_query("SELECT * FROM ipc WHERE level = 2", db_builder.conn)
    print(f"✓ Found {len(sections_df)} sections")
    
    # Test for each section
    for _, section in sections_df.iterrows():
        children_df = pd.read_sql_query(
            "SELECT COUNT(*) as count FROM ipc WHERE parent = ?", 
            db_builder.conn, 
            params=[section['symbol']]
        )
        print(f"   Section {section['symbol']}: {children_df['count'].iloc[0]} children")
    
    print("\n✅ Compatibility testing completed successfully!")
    print("📊 Database is ready for use with existing visualization code")
    
except Exception as e:
    print(f"❌ Compatibility test failed: {e}")

🧪 Testing compatibility with existing visualization...
✓ Basic query test passed
   Retrieved 10 test records
✓ All required columns present
✓ Found 8 sections
   Section A: 16 children
   Section B: 38 children
   Section C: 21 children
   Section D: 9 children
   Section E: 8 children
   Section F: 18 children
   Section G: 15 children
   Section H: 7 children

✅ Compatibility testing completed successfully!
📊 Database is ready for use with existing visualization code


## 7. Data Analysis and Insights

In [42]:
# Generate insights about the IPC 2025.01 data
print("📈 IPC 2025.01 Data Analysis")
print("=" * 50)

# Basic statistics
cursor = db_builder.conn.cursor()

# 1. Overall structure
cursor.execute('SELECT COUNT(*) FROM ipc')
total_entries = cursor.fetchone()[0]

cursor.execute('SELECT MIN(creation_date), MAX(creation_date) FROM ipc WHERE creation_date > 0')
date_range = cursor.fetchone()

print(f"📊 Overall Statistics:")
print(f"   Total IPC entries: {total_entries:,}")
print(f"   Date range: {date_range[0]} - {date_range[1]}")
print(f"   Time span: {(date_range[1] - date_range[0]) // 10000} years")

# 2. Technology sections
sections_query = '''
    SELECT symbol, symbol_short, title_en, size, size_percent 
    FROM ipc 
    WHERE level = 2 
    ORDER BY symbol
'''
sections_df = pd.read_sql_query(sections_query, db_builder.conn)

print(f"\n🏗️  Technology Sections ({len(sections_df)} total):")
for _, section in sections_df.iterrows():
    print(f"   {section['symbol']}: {section['title_en']} ({section['size']:,} groups, {section['size_percent']:.1f}%)")

# 3. Largest technology areas
largest_areas = pd.read_sql_query('''
    SELECT symbol_short, title_en, size, size_percent, level
    FROM ipc 
    WHERE level IN (3, 4) AND size > 0
    ORDER BY size DESC 
    LIMIT 10
''', db_builder.conn)

print(f"\n🥇 Top 10 Largest Technology Areas:")
for i, area in largest_areas.iterrows():
    level_name = "Class" if area['level'] == 3 else "Subclass"
    title = area['title_en'][:60] + "..." if len(area['title_en']) > 60 else area['title_en']
    print(f"   {i+1:2d}. {area['symbol_short']} ({level_name}): {area['size']:,} groups - {title}")

# 4. Evolution over time
evolution_query = '''
    SELECT 
        creation_date / 10000 as decade,
        COUNT(*) as new_entries,
        SUM(size) as total_groups
    FROM ipc 
    WHERE creation_date > 0
    GROUP BY creation_date / 10000
    ORDER BY decade
'''
evolution_df = pd.read_sql_query(evolution_query, db_builder.conn)

print(f"\n⏳ IPC Evolution by Decade:")
for _, period in evolution_df.iterrows():
    decade = int(period['decade'])
    print(f"   {decade}0s: {period['new_entries']:,} new entries, {period['total_groups']:,} total groups")

# 5. Depth analysis
depth_analysis = pd.read_sql_query('''
    SELECT level, COUNT(*) as count, AVG(size) as avg_descendants
    FROM ipc 
    GROUP BY level 
    ORDER BY level
''', db_builder.conn)

print(f"\n🔍 Classification Depth Analysis:")
for _, level_info in depth_analysis.iterrows():
    level = int(level_info['level'])
    level_name = {2: 'Sections', 3: 'Classes', 4: 'Subclasses', 5: 'Main Groups'}.get(level, f'Level {level}')
    print(f"   Level {level} ({level_name}): {level_info['count']:,} entries, avg {level_info['avg_descendants']:.1f} descendants")

print(f"\n📋 Database Summary:")
print(f"   🗄️  Database file: {config.OUTPUT_DB}")
print(f"   📏 File size: {Path(config.OUTPUT_DB).stat().st_size / (1024*1024):.1f} MB")
print(f"   🏷️  Version: IPC {config.IPC_VERSION}")
print(f"   🌐 Language: {config.IPC_LANGUAGE}")
print(f"   ⏰ Created: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📈 IPC 2025.01 Data Analysis
📊 Overall Statistics:
   Total IPC entries: 79,833
   Date range: 19680901 - 20250101
   Time span: 56 years

🏗️  Technology Sections (8 total):
   A: HUMAN NECESSITIES (9,863 groups, 12.4%)
   B: PERFORMING OPERATIONS; TRANSPORTING (18,397 groups, 23.0%)
   C: CHEMISTRY; METALLURGY (15,147 groups, 19.0%)
   D: TEXTILES; PAPER (3,300 groups, 4.1%)
   E: FIXED CONSTRUCTIONS (3,484 groups, 4.4%)
   F: MECHANICAL ENGINEERING; LIGHTING; HEATING; WEAPONS; BLASTING (9,605 groups, 12.0%)
   G: PHYSICS (9,560 groups, 12.0%)
   H: ELECTRICITY (10,469 groups, 13.1%)

🥇 Top 10 Largest Technology Areas:
    1. C07 (Class): 5,291 groups - ORGANIC CHEMISTRY
    2. H01 (Class): 3,995 groups - ELECTRIC ELEMENTS
    3. G01 (Class): 3,135 groups - MEASURING; TESTING
    4. C07C (Subclass): 2,979 groups - ACYCLIC OR CARBOCYCLIC COMPOUNDS
    5. H04 (Class): 2,914 groups - ELECTRIC COMMUNICATION TECHNIQUE
    6. A61 (Class): 2,871 groups - MEDICAL OR VETERINARY SCIENCE; HYGIENE

## 9. Summary and Next Steps

In [44]:
print("\n" + "=" * 60)
print("🎉 IPC Database Builder - COMPLETED SUCCESSFULLY!")
print("=" * 60)
print()
print("✅ What was accomplished:")
print("   📖 Parsed WIPO IPC 2025.01 XML file (20+ MB)")
print(f"   🗃️  Extracted {len(ipc_df):,} classification entries")
print("   🧮 Calculated hierarchical statistics and relationships")
print("   📊 Generated size percentages and normalized values")
print("   🗄️  Created optimized SQLite database with indexes")
print("   🔍 Verified data integrity and compatibility")
print()
print("🎯 Key Improvements over previous version:")
print("   • Updated to latest IPC 2025.01 classification")
print("   • Improved XML parsing with better error handling")
print("   • Enhanced statistics calculation algorithms")
print("   • Optimized database schema with proper indexes")
print("   • Full backward compatibility with existing visualizations")
print("   • Comprehensive data validation and verification")
print()
print("📊 Database Statistics:")
print(f"   • Total entries: {len(ipc_df):,}")
print(f"   • Sections: {len(ipc_df[ipc_df['level'] == 2]):,}")
print(f"   • Classes: {len(ipc_df[ipc_df['level'] == 3]):,}")
print(f"   • Subclasses: {len(ipc_df[ipc_df['level'] == 4]):,}")
print(f"   • Main groups: {len(ipc_df[ipc_df['level'] == 5]):,}")
print(f"   • Total hierarchy levels: {ipc_df['level'].max() - ipc_df['level'].min() + 1}")
print()
print("🔧 Technical Details:")
print(f"   • Source: {config.IPC_XML_FILE}")
print(f"   • Database: {config.OUTPUT_DB}")
print(f"   • Version: IPC {config.IPC_VERSION}")
print(f"   • Language: {config.IPC_LANGUAGE}")
print(f"   • Processing time: {time.time() - start_time:.1f} seconds")
print()
print("🚀 Next Steps:")
print("   1. Test the updated database with existing IPC Browser")
print("   2. Verify all visualizations work correctly")
print("   3. Consider adding French language support")
print("   4. Explore adding version comparison features")
print("   5. Implement automated updates for future IPC releases")
print()
print("✨ The IPC Browser is now ready with the latest 2025.01 data!")
print("" * 60)


🎉 IPC Database Builder - COMPLETED SUCCESSFULLY!

✅ What was accomplished:
   📖 Parsed WIPO IPC 2025.01 XML file (20+ MB)
   🗃️  Extracted 79,833 classification entries
   🧮 Calculated hierarchical statistics and relationships
   📊 Generated size percentages and normalized values
   🗄️  Created optimized SQLite database with indexes
   🔍 Verified data integrity and compatibility

🎯 Key Improvements over previous version:
   • Updated to latest IPC 2025.01 classification
   • Improved XML parsing with better error handling
   • Enhanced statistics calculation algorithms
   • Optimized database schema with proper indexes
   • Full backward compatibility with existing visualizations
   • Comprehensive data validation and verification

📊 Database Statistics:
   • Total entries: 79,833
   • Sections: 8
   • Classes: 132
   • Subclasses: 654
   • Main groups: 7,630
   • Total hierarchy levels: 13

🔧 Technical Details:
   • Source: /home/jovyan/mtc-patent-analytics/ipc-browser/ipc/EN_ipc_sch